In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,19 mins 32 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.3
H2O_cluster_version_age:,12 days
H2O_cluster_name:,bonay
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.51 Gb
H2O_cluster_total_cores:,11
H2O_cluster_allowed_cores:,11
H2O_cluster_status:,"locked, healthy"


In [2]:
from h2o.automl import H2OAutoML

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering

In [5]:
file_path = 'dataset/n_vima.csv'
df = pd.read_csv(file_path, quotechar='"', delimiter=";")
df['STOCK_ACTUAL'] = df['STOCK_ACTUAL'].str.replace(',', '.').astype(float)
df['TOTAL_CANTIDAD_MOVS'] = df['TOTAL_CANTIDAD_MOVS'].str.replace(',', '.').astype(float)
df['PMP'] = df['PMP'].str.replace(',', '.').astype(float)
df['TOTAL_IMPORTE_MOVS'] = df['TOTAL_IMPORTE_MOVS'].str.replace(',', '.').astype(float)
label_encoder = LabelEncoder()
df['ARTICULO'] = label_encoder.fit_transform(df['ARTICULO'])
df['TIPO_MATERIAL'] = label_encoder.fit_transform(df['TIPO_MATERIAL'])
df['CODIGO_FAMILIA'] = label_encoder.fit_transform(df['CODIGO_FAMILIA'])
df.drop(columns=['EMPRESA','PESO_NETO','MULTIPRESENTACION','ARTICULO','CODIGO_FAMILIA'], inplace=True)

train_data = df.sample(frac=0.8, random_state=0)
test_data = df.drop(train_data.index)

In [ ]:
train = h2o.H2OFrame(train_data)
test = h2o.H2OFrame(test_data)


x = list(train.columns)

aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=600, )
aml.train(x=x, training_frame=train)

In [ ]:
aml.leaderboard

In [ ]:
perf = aml.leader.model_performance(test)

In [ ]:
aml.predict(test)

In [ ]:

perf.show()

In [ ]:
perf.confusion_matrix()

In [ ]:
#We need to get the predictions from the model: accuracy, precision, recall, f1-score
predictions = aml.predict(test)
predictions = predictions.as_data_frame()
predictions = predictions['predict'].values
test = test.as_data_frame()
test = test['CATEGORIA_FACTURA'].values


In [ ]:
#show the results
print("Accuracy: ", accuracy_score(test, predictions))
print("Precision: ", precision_score(test, predictions, average='weighted'))
print("Recall: ", recall_score(test, predictions, average='weighted'))
print("F1-Score: ", f1_score(test, predictions, average='weighted'))
print(classification_report(test, predictions))
print(confusion_matrix(test, predictions))


In [ ]:
perf.hit_ratio_table()